# Load Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from sklearn.utils import shuffle

# Load Data

Firstly, the data prepared in Notebook 1 are loaded (data combined.csv), including dropping missing values (just in case any creeped in!).

Next, we specify the inputs and outputs. Note that to_drop contains any columns that are not to be used in inputs. Inputs will only contain molecular descriptors, M, T and p.
Outputs will be density values.

Output is divided by 1000 to convert the unit from kg/m3 to g/cm3 (this will be important in PINN).

In [ ]:
df = pd.read_csv('./Data/data combined.csv', index_col = 0).dropna(axis = 0)
print('Loaded dataset:')
display(df)

extra_tags = ['M g/mol', 'T / K', 'p / MPa']
to_drop = ['Dataset ID', 'IL ID', 'Cation', 'Anion', 'Cationic family', 'Anionic family',
           'Excluded IL', 'Accepted dataset', 'T / K', 'p / MPa', 'ρ / kg/m3', 'SWMLR (v0) + FFANN (f)',
           'SWMLR (v0) + FFANN (f).1', 'FFANN (v0) + FFANN (f)', 'FFANN (v0) + FFANN (f).1',
           'LSSVM (v0) + FFANN (f)', 'LSSVM (v0) + FFANN (f).1', 'M g/mol']
molecular_descriptors = df.drop(to_drop, axis = 1).columns
inputs = np.hstack((molecular_descriptors, 
                    extra_tags))

outputs = ['ρ / kg/m3']

print(f'\n\nInputs {len(inputs)}:\n{inputs}')
print(f'\n\nOutputs {len(outputs)}:\n{outputs}')

X = df[inputs]
y = df[outputs]/1000

print('\n\nInput data:')
display(X)

print('\n\nOutput data:')
display(y)

# Scale and Split Data

The code defines two functions, scale and descale, for normalising and denormalising data. The scale function adjusts the input data by subtracting a shift value and dividing by a factor for each column, effectively normalising the data. The descale function reverses this process, restoring the original data values by multiplying by the factor and adding the shift.

In the preparation section, a dictionary named scaler is created to store the shift and factor values for both input features (X) and output values (y). These values are used to scale the inputs and outputs into a range defined by scaler_range.

The scaled data is then stored in the dataset dictionary, and any columns with all NaN values are dropped. The dataset is split into training and testing sets using a 15% test size, with shapes of the resulting sets printed to confirm the split.

In [ ]:
def scale(data_in, shift, factor):
    """
    Scale the input data by shifting and dividing by a factor.
    
    Parameters:
    data_in (array-like or DataFrame): The input data to be scaled.
    shift (Series or DataFrame): The shift values for each column in the data.
    factor (Series or DataFrame): The factor values for each column in the data.
    
    Returns:
    DataFrame: The scaled data.
    """
    # Convert the input data to a DataFrame
    data_in = pd.DataFrame(data_in)
    
    # Scale the data by subtracting the shift and dividing by the factor for each column
    return (data_in - shift[data_in.columns]) / factor[data_in.columns]

def descale(data_in, shift, factor):
    """
    Descale the input data by multiplying by a factor and adding a shift.
    
    Parameters:
    data_in (array-like or DataFrame): The input data to be descaled.
    shift (Series or DataFrame): The shift values for each column in the data.
    factor (Series or DataFrame): The factor values for each column in the data.
    
    Returns:
    DataFrame: The descaled data.
    """
    # Convert the input data to a DataFrame
    data_in = pd.DataFrame(data_in)
    
    # Descale the data by multiplying by the factor and adding the shift for each column
    return data_in * factor[data_in.columns] + shift[data_in.columns]

# Prepare scaler
scaler = {}
scaler_range = (0, 1)
scaler['X_shift'] = X.min(axis=0)
scaler['X_factor'] = X.max(axis=0) - X.min(axis=0)
scaler['y_shift'] = y.min(axis=0)
scaler['y_factor'] = y.max(axis=0) - y.min(axis=0)

# Scale inputs X and output y
dataset = {}
dataset['X_scaled'] = scale(X, scaler['X_shift'], scaler['X_factor']).dropna(axis = 1, how = 'all')
dataset['Y_scaled'] = scale(y, scaler['y_shift'], scaler['y_factor'])
print('Scaled input data size:', dataset['X_scaled'].shape)
print('Scaled outputs data size:', dataset['Y_scaled'].shape)

# Split data into training and testing
dataset['X_train_scaled'], dataset['X_test_scaled'], dataset['Y_train_scaled'], dataset['Y_test_scaled'] = train_test_split(dataset['X_scaled'], dataset['Y_scaled'], test_size=0.15, random_state = 23)
print('Training size:', dataset['X_train_scaled'].shape, dataset['Y_train_scaled'].shape)
print('Testing size:', dataset['X_test_scaled'].shape, dataset['Y_test_scaled'].shape)

# Define Model and Training Functions

This code defines a physics-informed neural network (PINN) model using PyTorch and includes functions for training, evaluating, and tracking the progress of the model.

The PINNModel_torch class defines the architecture of the PINN model, which includes two hidden layers with 100 neurons each and multiple output layers for predicting density (predicted_density), base density (rho_0_pred), compressibility factor (k_pred), and specific volume (v_pred). The forward pass method applies ReLU activation functions to the outputs of the hidden layers and computes the final predicted values.

The FTEoS function calculates the density using the Fluid Thermodynamic Equation of State (FTEoS) based on input parameters such as molecular weight (M), temperature (T), pressure (p), base density (rho_0), compressibility factor (k), and specific volume (v). The calculated density is scaled back to match the original data scale.

The loss_fn function calculates the combined loss for the PINN model by computing the Mean Squared Error (MSE) between the model predictions and true values for both the neural network predictions and the FTEoS predictions. This combined loss function helps in guiding the training of the model.

The closure function performs a single optimization step, including zeroing the gradients, computing the loss, performing backpropagation, and updating the model parameters.

The get_metrics function calculates various performance metrics for the model, such as training and testing loss, Mean Squared Error (MSE), and Mean Absolute Error (MAE), and returns these metrics as a list.

The print_progress function prints the progress of the training process, including the current epoch's training and testing loss, MSE, and MAE. It highlights epochs where a new best test loss is achieved and includes an asterisk for such epochs.

In [ ]:
class PINNModel_torch(nn.Module):
    """
    A simple physics-informed neural network (PINN) model using PyTorch.
    
    Parameters:
    input_dim (int): The number of input features.
    """
    def __init__(self, input_dim):
        super(PINNModel_torch, self).__init__()
        # Define the first fully connected layer with 100 neurons
        self.dense1 = nn.Linear(input_dim, 100)
        # Define the second fully connected layer with 100 neurons
        self.dense2 = nn.Linear(100, 100)
        # Define the output layers for different predicted values
        self.dense_density = nn.Linear(100, 1)  # Predicted density
        self.dense_rho0 = nn.Linear(100, 1)     # Predicted rho_0
        self.dense_k = nn.Linear(100, 1)        # Predicted k
        self.dense_v = nn.Linear(100, 1)        # Predicted v

    def forward(self, x):
        """
        Forward pass through the network.
        
        Parameters:
        x (Tensor): Input tensor.
        
        Returns:
        Tuple[Tensor, Tensor, Tensor, Tensor]: Output tensors for predicted values.
        """
        # Apply ReLU activation function to the output of the first layer
        x = torch.relu(self.dense1(x))
        # Apply ReLU activation function to the output of the second layer
        x = torch.relu(self.dense2(x))
        # Generate the final outputs
        predicted_density = self.dense_density(x)
        rho_0_pred = self.dense_rho0(x)
        k_pred = self.dense_k(x)
        v_pred = self.dense_v(x)
        return predicted_density, rho_0_pred, k_pred, v_pred

def FTEoS(M, T, p, rho_0, k, v, p0=0.1, R=8.314):
    """
    Calculate the density using the FTEoS (Fluid Thermodynamic Equation of State).
    
    Parameters:
    M (Tensor): Molecular weight.
    T (Tensor): Temperature.
    p (Tensor): Pressure.
    rho_0 (Tensor): Base density.
    k (Tensor): Compressibility factor.
    v (Tensor): Specific volume.
    p0 (float): Reference pressure (default is 0.1).
    R (float): Gas constant (default is 8.314).
    
    Returns:
    Tensor: Scaled calculated density.
    """
    # Flatten tensors
    rho_0 = torch.flatten(rho_0)
    k = torch.flatten(k)
    v = torch.flatten(v)
    # Convert scaled values to original values using factors and shifts
    M = M * scaler['X_factor']['M g/mol'] + scaler['X_shift']['M g/mol']
    T = T * scaler['X_factor']['T / K'] + scaler['X_shift']['T / K']
    p = p * scaler['X_factor']['p / MPa'] + scaler['X_shift']['p / MPa']
    # Calculate the density
    density_calc = rho_0 + (1/k) * torch.log10(((M*k)/(v*R*T))*(p-p0)+1)
    # Scale the calculated density
    density_calc_scaled = (density_calc - scaler['y_shift']['ρ / kg/m3'])/scaler['y_factor']['ρ / kg/m3'] 
    return density_calc_scaled

def loss_fn(model, X, y, M, T, p):
    """
    Calculate the combined loss for the PINN model.
    
    Parameters:
    model (nn.Module): The PINN model.
    X (Tensor): Input features.
    y (Tensor): True density values.
    M (Tensor): Molecular weight.
    T (Tensor): Temperature.
    p (Tensor): Pressure.
    
    Returns:
    Tensor: Combined loss value.
    """
    # Get model predictions
    y_pred, rho_0_pred, k_pred, v_pred = model(X)
    # Calculate MLP loss
    mse_MLP = nn.functional.mse_loss(y_pred, y)
    # Calculate FTEoS predictions and PINN loss
    y_pred_EOS = FTEoS(M, T, p, rho_0_pred, k_pred, v_pred)
    mse_PINN = nn.functional.mse_loss(y_pred_EOS, y)
    # Return combined loss
    return 0.5 * mse_MLP + 0.5 * mse_PINN

def closure():
    """
    Perform a single optimization step for the model.
    
    Returns:
    Tensor: The loss value.
    """
    # Zero the gradients of the optimizer
    optimizer.zero_grad()
    # Compute the loss
    loss = loss_fn(model, x_batch, y_batch, M_batch, T_batch, p_batch)
    loss = loss.mean()
    # Backward pass: compute gradient of the loss with respect to parameters
    loss.backward()
    return loss

def get_metrics(model, X_train, X_test, y_train, y_test, loss_fn):
    """
    Calculate and return various performance metrics for the model.
    
    Parameters:
    model (nn.Module): The trained model.
    X_train (Tensor): Training input data.
    X_test (Tensor): Testing input data.
    y_train (Tensor): Training target data.
    y_test (Tensor): Testing target data.
    loss_fn (function): Loss function.
    
    Returns:
    list: A list of metrics [train loss, test loss, train MSE, test MSE, train MAE, test MAE].
    """
    # Use MSE loss function without reduction
    mse_loss = torch.nn.MSELoss(reduction='none')
    # Predict on the training data
    y_pred_train = model(X_train)[0]
    # Compute the training loss
    loss_train = mse_loss(y_pred_train, y_train)
    # Predict on the testing data
    y_pred_test = model(X_test)[0]
    # Compute the testing loss
    loss_test = mse_loss(y_pred_test, y_test)
    # Calculate Mean Squared Error (MSE) for training data
    MSE = torch.mean((y_pred_train - y_train) ** 2)
    # Calculate Mean Absolute Error (MAE) for training data
    MAE = torch.mean(torch.abs(y_pred_train - y_train))
    # Calculate Mean Squared Error (MSE) for testing data
    MSE_test = torch.mean((y_pred_test - y_test) ** 2)
    # Calculate Mean Absolute Error (MAE) for testing data
    MAE_test = torch.mean(torch.abs(y_pred_test - y_test))
    # Return the computed metrics as a list
    return [loss_train.mean().item(), 
            loss_test.mean().item(), 
            MSE.item(), 
            MSE_test.item(),
            MAE.item(),
            MAE_test.item()]

def print_progress(epoch, loss_info, best_loss):
    """
    Print the progress of training, including various metrics.
    
    Parameters:
    epoch (int): The current epoch number.
    loss_info (DataFrame): DataFrame containing the loss information.
    best_loss (float): The best loss achieved so far.
    
    Returns:
    None
    """
    # Check if the current epoch's test loss is better than the best loss
    if loss_info.loc[epoch, 'loss test'] < best_loss:
        # Print progress with an asterisk indicating a new best test loss
        print('{}   {:.3e}/{:.3e}   {:.3e}/{:.3e}   {:.3e}/{:.3e} *'.format(epoch,
            loss_info.loc[epoch, 'loss train'],
            loss_info.loc[epoch, 'loss test'],
            loss_info.loc[epoch, 'MSE train'], 
            loss_info.loc[epoch, 'MSE test'],
            loss_info.loc[epoch, 'MAE train'], 
            loss_info.loc[epoch, 'MAE test']))
    else:
        # Print progress without an asterisk
        print('{}   {:.3e}/{:.3e}   {:.3e}/{:.3e}   {:.3e}/{:.3e}    {}'.format(epoch,
            loss_info.loc[epoch, 'loss train'],
            loss_info.loc[epoch, 'loss test'],
            loss_info.loc[epoch, 'MSE train'], 
            loss_info.loc[epoch, 'MSE test'],
            loss_info.loc[epoch, 'MAE train'], 
            loss_info.loc[epoch, 'MAE test'],
            epoch - last_loss))

# Train Model
This code details the training and evaluation process for a Physics-Informed Neural Network (PINN) model using PyTorch.

First, the code sets the computation device to GPU ('cuda') if available, for efficient training. The PINNModel_torch is initialized with the input feature dimension and moved to the GPU. The Adam optimizer is set up with a specified learning rate to update the model parameters during training.

Training and testing data are converted into PyTorch tensors and moved to the GPU. A DataFrame is initialized to store various loss metrics, including training and testing loss, Mean Squared Error (MSE), and Mean Absolute Error (MAE).

Training parameters such as the number of epochs, early stopping patience, last improved epoch, batch size, and best loss are defined. The training loop begins by shuffling the training data for each epoch to ensure robust training. Mini-batch training is performed by dividing the data into batches and performing optimization steps on each batch. During each optimization step, the model parameters are updated based on the loss calculated using the closure function.

After each epoch, performance metrics are calculated and stored. The training progress is printed, and the model is saved if the current test loss is the best seen so far. Early stopping is implemented to terminate training if no improvement in test loss is observed for a specified number of epochs (patience), preventing overfitting and saving computational resources.

In [ ]:
# Set the device to GPU if available
device = 'cuda'

# Initialize the PINN model and move it to the specified device
#model = PINNModel_torch(dataset['X_train_scaled'].shape[1]).to(device)

# Set up the optimizer (Adam) with the model parameters and a learning rate
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Convert training and testing data to PyTorch tensors and move to the specified device
x_train = torch.tensor(dataset['X_train_scaled'].values, dtype=torch.float32, device=device)
y_train = torch.tensor(dataset['Y_train_scaled'].values, dtype=torch.float32, device=device)
x_test = torch.tensor(dataset['X_test_scaled'].values, dtype=torch.float32, device=device)
y_test = torch.tensor(dataset['Y_test_scaled'].values, dtype=torch.float32, device=device)

# Initialize a DataFrame to store loss information
loss_info = pd.DataFrame(columns=['loss train', 'loss test', 'MSE train', 'MSE test', 'MAE train', 'MAE test'])

# Set training parameters
epochs = 100000
patience = 150
last_loss = 0
batch_size = 50
#best_loss = np.inf

# Training loop
for epoch in range(epochs):
    # Shuffle the training data
    x_shuffle, y_shuffle = shuffle(x_train, y_train)
    
    # Mini-batch training
    for i in range(0, x_train.shape[0], batch_size):
        # Create mini-batches for training
        x_batch = x_shuffle[i:i + batch_size]
        y_batch = y_shuffle[i:i + batch_size]
        # Extract the last three columns as p_batch, T_batch, and M_batch
        p_batch = x_batch[:, -1]
        T_batch = x_batch[:, -2]
        M_batch = x_batch[:, -3]
        
        # Perform a single optimization step
        optimizer.step(closure)
        
    # Calculate and store metrics for the current epoch
    loss_info.loc[epoch, :] = get_metrics(model, x_train, x_test, y_train, y_test, loss_fn)
    # Print progress for the current epoch
    print_progress(epoch, loss_info, best_loss)
    
    # Save the model if the current test loss is the best seen so far
    if loss_info.loc[epoch, 'loss test'] < best_loss:
        torch.save(model.state_dict(), './Models/model_PINN.pt')
        best_loss = loss_info.loc[epoch, 'loss test']
        last_loss = epoch
        
    # Early stopping if no improvement in test loss for 'patience' epochs
    if epoch - last_loss >= patience:
        print('Terminated')
        break


In [ ]:
import warnings
warnings.filterwarnings("ignore")